In [2]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

In [4]:
train_path = r'C:\Users\indra\Downloads\learning-of-structured-data-fhws-ws2324\train\train'


train_file_names = [f for f in os.listdir(train_path) if f.endswith('.csv')]

train_sequences = []
labels = []

label_encoder = LabelEncoder()

for file_name in train_file_names:
    # Extract the label, assuming that the label is the part after the "_" character and before the "." character
    label = file_name.split('_')[-1].split('.')[0]
    labels.append(label)

    file_path = os.path.join(train_path, file_name)

    num_rows = len(pd.read_csv(file_path, usecols=[0])) // 2  # Use just the first column to determine the number of rows
    df_half = pd.read_csv(file_path, nrows=num_rows, usecols=range(75)) # Take the first half of the rows of the CSV file

    train_sequence = df_half.values.astype('float32')
    train_sequence = pd.DataFrame(train_sequence)

    train_sequences.append(train_sequence)

# Encode labels with value between 0 and n_classes-1.
encoded_labels = label_encoder.fit_transform(labels)

train_sequences[0]

,0,1,2,3,4,5,6,7,8,9,...,65,66,67,68,69,70,71,72,73,74
0,267.0,118.0,1.0,279.0,186.0,1.0,209.0,201.0,1.0,183.0,...,0.0,-0.363440,-0.021049,0.0,-0.363440,-0.021049,0.0,-0.363440,-0.021049,0.0
1,266.0,119.0,1.0,278.0,187.0,1.0,209.0,202.0,1.0,184.0,...,0.0,-0.182489,0.001790,0.0,-0.182489,0.001790,0.0,-0.182489,0.001790,0.0
2,265.0,120.0,1.0,277.0,188.0,1.0,209.0,203.0,1.0,184.0,...,0.0,-0.118099,0.006548,0.0,-0.118099,0.006548,0.0,-0.118099,0.006548,0.0
3,265.0,121.0,1.0,277.0,189.0,1.0,210.0,202.0,1.0,184.0,...,0.0,-0.110839,0.012986,0.0,-0.110839,0.012986,0.0,-0.110839,0.012986,0.0
4,265.0,122.0,1.0,277.0,190.0,1.0,210.0,203.0,1.0,185.0,...,0.0,-0.073655,-0.028735,0.0,-0.073655,-0.028735,0.0,-0.073655,-0.028735,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340,280.0,92.0,1.0,283.0,187.0,1.0,218.0,197.0,1.0,200.0,...,0.0,-2.027836,0.012127,0.0,-2.027836,0.012127,0.0,-2.027836,0.012127,0.0
341,280.0,91.0,1.0,283.0,190.0,1.0,218.0,197.0,1.0,199.0,...,0.0,-2.040888,0.011828,0.0,-2.040888,0.011828,0.0,-2.040888,0.011828,0.0
342,280.0,90.0,1.0,283.0,190.0,1.0,218.0,197.0,1.0,198.0,...,0.0,-2.159388,0.009583,0.0,-2.159388,0.009583,0.0,-2.159388,0.009583,0.0
343,280.0,89.0,1.0,283.0,190.0,1.0,218.0,197.0,1.0,198.0,...,0.0,-1.916025,0.012349,0.0,-1.916025,0.012349,0.0,-1.916025,0.012349,0.0


In [5]:
test_path = train_path = r'C:\Users\indra\Downloads\learning-of-structured-data-fhws-ws2324\test\test'

test_file_names = [f for f in os.listdir(test_path) if f.endswith('.csv')]

# Sort the test file names in ascending order
test_file_names.sort(key=lambda x: int(x.split('.')[0]))

test_sequences = []

for file_name in test_file_names:

    file_path = os.path.join(test_path, file_name)

    num_rows = len(pd.read_csv(file_path, usecols=[0])) // 2
    df_half = pd.read_csv(file_path, nrows=num_rows, usecols=range(75))

    test_sequence = df_half.values.astype('float32')
    test_sequence = pd.DataFrame(test_sequence)

    test_sequences.append(test_sequence)

test_sequences[0]

,0,1,2,3,4,5,6,7,8,9,...,65,66,67,68,69,70,71,72,73,74
0,308.0,93.0,1.0,310.0,163.0,1.0,248.0,167.0,1.0,210.0,...,0.0,-0.744930,0.402878,0.0,-0.744930,0.402878,0.0,-0.744930,0.402878,0.0
1,307.0,91.0,1.0,308.0,163.0,1.0,246.0,167.0,1.0,208.0,...,0.0,-1.625237,0.235531,0.0,-1.625237,0.235531,0.0,-1.625237,0.235531,0.0
2,306.0,89.0,1.0,307.0,163.0,1.0,247.0,167.0,1.0,207.0,...,0.0,-3.048215,0.607103,0.0,-3.048215,0.607103,0.0,-3.048215,0.607103,0.0
3,305.0,88.0,1.0,306.0,163.0,1.0,246.0,167.0,1.0,206.0,...,0.0,-3.110481,0.680721,0.0,-3.110481,0.680721,0.0,-3.110481,0.680721,0.0
4,304.0,87.0,1.0,305.0,163.0,1.0,245.0,167.0,1.0,206.0,...,0.0,-2.775089,1.236649,0.0,-2.775089,1.236649,0.0,-2.775089,1.236649,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274,291.0,95.0,1.0,301.0,166.0,1.0,242.0,167.0,1.0,203.0,...,0.0,-16.059616,-130.833481,0.0,-16.059616,-130.833481,0.0,-16.059616,-130.833481,0.0
275,291.0,95.0,1.0,301.0,166.0,1.0,242.0,167.0,1.0,203.0,...,0.0,-16.059616,-130.833481,0.0,-16.059616,-130.833481,0.0,-16.059616,-130.833481,0.0
276,291.0,95.0,1.0,301.0,166.0,1.0,242.0,167.0,1.0,202.0,...,0.0,-16.059616,-130.833481,0.0,-16.059616,-130.833481,0.0,-16.059616,-130.833481,0.0
277,291.0,96.0,1.0,301.0,166.0,1.0,242.0,167.0,1.0,201.0,...,0.0,-16.059616,-130.833481,0.0,-16.059616,-130.833481,0.0,-16.059616,-130.833481,0.0


In [ ]:
# if the confidence is 0, set the previous 2 values to NAN
for df in train_sequences:
    for row in range(len(df)):
        for col in range(2, 75, 3):
            if df.iloc[row, col] == 0:
                df.iloc[row, col-2:col] = np.nan

for df in test_sequences:
    for row in range(len(df)):
        for col in range(2, 75, 3):
            if df.iloc[row, col] == 0:
                df.iloc[row, col-2:col] = np.nan

train_sequences[0]

In [ ]:
# merge multiple dataframes in the training dataset into one dataframe
train_df = pd.concat(train_sequences)

# reset the index of the merged dataframe
train_df.reset_index(drop=True, inplace=True)

train_df

In [ ]:
# drop keypoints with confidence of 0 over confidence_threshold
key_points_mapping = {
        2: "Nose", 5: "Neck", 8: "RShoulder", 11: "RElbow", 14: "RWrist",
        17: "LShoulder", 20: "LElbow", 23: "LWrist", 26: "MidHip", 29: "RHip",
        32: "RKnee", 35: "RAnkle", 38: "LHip", 41: "LKnee", 44: "LAnkle",
        47: "REye", 50: "LEye", 53: "REar", 56: "LEar", 59: "LBigToe",
        62: "LSmallToe", 65: "LHeel", 68: "RBigToe", 71: "RSmallToe", 74: "RHeel",
    }

# set confidence threshold
confidence_threshold = 0.9

# store column indices to drop
columns_to_drop = []

# iterate over confidence columns
for col in range(2, 75, 3):
    # calculate the proportion of 0 confidence
    zero_confidence_ratio = (train_df[col] == 0).mean()
    # if the ratio is over 90%, drop the corresponding x,y coordinate columns and confidence column
    if zero_confidence_ratio > confidence_threshold:
        columns_to_drop.extend([col-2, col-1, col])
        print(f'Delete {col} : {key_points_mapping[col]}')

# drop columns_to_drop from train_sequences and reset column names
for df in train_sequences:
    df.drop(columns_to_drop, axis=1, inplace=True)

# drop columns_to_drop from test_sequences and reset column names
for df in test_sequences:
    df.drop(columns_to_drop, axis=1, inplace=True)

train_sequences[0]

In [ ]:
# drop the last column in every three columns in train_sequences
for df in train_sequences:
    df.drop(columns=df.columns[2::3], inplace=True)

for df in test_sequences:
    df.drop(columns=df.columns[2::3], inplace=True)

train_sequences[0]

In [ ]:
# normalize the data in train_sequences so that the mean of each keypoint is 0 and the standard deviation is 1
for df in train_sequences:
    df.iloc[:, :] = (df.iloc[:, :] - df.iloc[:, :].mean()) / df.iloc[:, :].std()

for df in test_sequences:
    df.iloc[:, :] = (df.iloc[:, :] - df.iloc[:, :].mean()) / df.iloc[:, :].std()

train_sequences[0]

In [ ]:
# fill the NAN values in train_sequences with 0
for df in train_sequences:
    df.fillna(0, inplace=True)

for df in test_sequences:
    df.fillna(0, inplace=True)

train_sequences[0]

In [ ]:
# truncate each df in train_sequences to 150 rows, if there are less than 150 rows, fill with 0
for i, df in enumerate(train_sequences):
    if len(df) > 150:
        train_sequences[i] = df.iloc[:150, :]
    else:
        train_sequences[i] = df.reindex(range(150), fill_value=0)

for i, df in enumerate(test_sequences):
    if len(df) > 150:
        test_sequences[i] = df.iloc[:150, :]
    else:
        test_sequences[i] = df.reindex(range(150), fill_value=0)

train_sequences[0]

In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(LSTMModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim
        self.lstm = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).to(x.device)
        c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).to(x.device)
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
        out = self.fc(out[:, -1, :])
        return out


In [ ]:
# Bidirectional LSTM

class BiLSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim, dropout_prob=0.3):
        super(BiLSTMModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim

        # A bidirectional LSTM layer
        self.lstm = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True, dropout=dropout_prob, bidirectional=True)

        self.fc = nn.Linear(hidden_dim*2, output_dim)

    def forward(self, x):
        h0 = torch.zeros(self.layer_dim*2, x.size(0), self.hidden_dim).to(x.device)
        c0 = torch.zeros(self.layer_dim*2, x.size(0), self.hidden_dim).to(x.device)

        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))

        # Take the output of the last time step for classification
        out = self.fc(out[:, -1, :])
        return out


In [ ]:
# transform data into tensors
X_tensor = torch.tensor([df.values for df in train_sequences], dtype=torch.float32)
y_tensor = torch.tensor(encoded_labels, dtype=torch.long)

# split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)

# create data loaders
train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)

train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=32, shuffle=False)


In [ ]:
input_dim = 34
hidden_dim = 256
layer_dim = 6
output_dim = 5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# model = LSTMModel(input_dim, hidden_dim, layer_dim, output_dim)
model = BiLSTMModel(input_dim, hidden_dim, layer_dim, output_dim)

model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 50

best_acc = 0.0
best_model = None
for epoch in range(num_epochs):
    model.train()
    train_loss, train_acc = 0, 0
    for i, (X, y) in enumerate(train_loader):
        X, y = X.to(device), y.to(device)
        optimizer.zero_grad()
        outputs = model(X)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        train_acc += (outputs.argmax(1) == y).sum().item()

    train_loss /= len(train_loader.dataset)
    train_acc /= len(train_loader.dataset)

    model.eval()
    val_loss, val_acc = 0, 0
    with torch.no_grad():
        for X, y in val_loader:
            X, y = X.to(device), y.to(device)
            outputs = model(X)
            loss = criterion(outputs, y)
            val_loss += loss.item()
            val_acc += (outputs.argmax(1) == y).sum().item()

    val_loss /= len(val_loader.dataset)
    val_acc /= len(val_loader.dataset)

    # if the current model has a higher validation accuracy, save it
    if val_acc > best_acc:
        best_acc = val_acc
        best_model = model.state_dict()

    print(f'Epoch {epoch}/{num_epochs}: Train Loss: {train_loss}, Train Acc: {train_acc}, Val Loss: {val_loss}, Val Acc: {val_acc}')

print(f'The best model has a validation accuracy of {best_acc}')


In [ ]:
# predict
X_test = torch.tensor([df.values for df in test_sequences], dtype=torch.float32).to(device)

# model.load_state_dict(best_model)
model.eval()
predictions = []
with torch.no_grad():
    for X in X_test:
        X = X.unsqueeze(0)
        outputs = model(X)
        _, predicted = torch.max(outputs.data, 1)
        predictions.append(predicted.item())

# Save predictions to CSV
predictions_df = pd.DataFrame(predictions, columns=['action'])
predictions_df.insert(0, 'id', range(len(predictions_df)))
predictions_df.to_csv('submission.csv', index=False)